In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
MNIST_TRAIN = np.load('./MNIST_train_40000.npz')
print(MNIST_TRAIN.keys())

['train_labels', 'train_images']


In [3]:
MNIST_TRAIN['train_images'].shape

(40000, 28, 28)

In [4]:
labels_train = MNIST_TRAIN['train_labels']
labels_train.shape

(40000,)

In [5]:
labels_one_hot_train = pd.get_dummies(labels_train)
labels_one_hot_train.head()

,0,1,2,3,4,5,6,7,8,9
0,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0


In [6]:
images_train = MNIST_TRAIN['train_images']
images_train.shape

(40000, 28, 28)

In [7]:
images_train_reshaped = images_train.reshape((40000, 28*28))
images_train_reshaped.shape

(40000, 784)

In [8]:
# layer 1
x = tf.placeholder(dtype = 'float32',shape=[40000,784])
y1 = tf.placeholder(dtype = 'float32', shape=[40000, 20])
w1 = tf.Variable(tf.truncated_normal([784, 20], stddev=1))
b1 = tf.Variable(tf.truncated_normal([1, 20], stddev=1))

# layer 2
w2 = tf.Variable(tf.truncated_normal([20, 10], stddev=1))
b2 = tf.Variable(tf.truncated_normal([1, 10], stddev=1))
Y2 = tf.placeholder(dtype= 'float32', shape=[40000, 10])

In [9]:
y2 = tf.matmul((tf.matmul(x, w1)) + b1, w2) + b2

In [36]:
# configure test data
MNIST_TEST = np.load('./MNIST_test_10000.npz')
labels_test = MNIST_TEST['test_labels']
labels_one_hot_test = pd.get_dummies(labels_test)
images_test = MNIST_TEST['test_images']
images_test_reshaped = images_test.reshape((10000, 28*28))
print(images_test_reshaped.shape)

(10000, 784)


In [37]:
x_test = tf.placeholder(dtype = 'float32',shape=[10000,784])
Y_TEST = tf.placeholder(dtype= 'float32', shape=[10000, 10])
y_test = tf.matmul((tf.matmul(x_test, w1)) + b1, w2) + b2

In [38]:
ce_train = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y2, labels=Y2))
ce_test = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_test, labels=Y_TEST))

In [39]:
optimizer = tf.train.AdamOptimizer().minimize(ce)
init = tf.global_variables_initializer()

sess = tf.Session()
MAXSTEPS = 1000
MOD=100
#initialize all variables
sess.run(init)

In [40]:
for step in range(MAXSTEPS+1):
    (_, ce_train_opt, ce_test_opt) = sess.run([optimizer, ce_train, ce_test],feed_dict={x: images_train_reshaped, x_test: images_test_reshaped, Y2: labels_one_hot_train, Y_TEST: labels_one_hot_test})
    if (step % MOD) == 0:
        print(step, ce_train_opt, ce_test_opt)

0 58.1786 58.6521
100 18.4088 18.7466
200 9.12069 9.47462
300 5.77458 6.11321
400 4.20324 4.51206
500 3.32567 3.61362
600 2.7712 3.04649
700 2.38469 2.6593
800 2.09901 2.37416
900 1.87755 2.15156
1000 1.69748 1.96963


In [42]:
loss_p_train = tf.nn.softmax(y2)
loss_p_test = tf.nn.softmax(y_test)

In [43]:
(yp_train) = sess.run(loss_p_train, feed_dict={x: images_train_reshaped})

In [44]:
(yp_test) = sess.run(loss_p_test, feed_dict={x_test: images_test_reshaped})

In [45]:
err_train=(yp_train.argmax(axis=1) != labels_one_hot_train.values.argmax(axis=1)).sum()
err_test=(yp_test.argmax(axis=1) != labels_one_hot_test.values.argmax(axis=1)).sum()

In [46]:
err_train = err_train / labels_one_hot_train.shape[0]
print("error_rate: ", err_train)

error_rate:  0.165475


In [47]:
err_test = err_test / labels_one_hot_test.shape[0]
print("error_rate: ", err_test)

error_rate:  0.1805
